 ## HDA-7: fMRI Functional Connectivity Feature Extraction
 This script:
 - (Optionally) downloads an OpenNeuro dataset (ds002785) via DataLad
 - Runs fMRIPrep via Docker (manual command provided)
 - Extracts ROI timeseries, computes connectivity metrics (Nilearn + NetworkX)
 - Saves a `secondary_dataset.csv`

In [ ]:
import os
import subprocess
import glob
import warnings
import numpy as np
import pandas as pd
from nilearn.maskers import NiftiLabelsMasker
from nilearn import datasets
from nilearn.connectome import ConnectivityMeasure
import networkx as nx

warnings.filterwarnings('ignore')

# ========== 1) Install lightweight Python deps ==========
def install_requirements():
    try:
        subprocess.run(
            ["pip", "install", "--quiet", "nilearn", "nibabel", "numpy", "pandas", "networkx", "scikit-learn", "tqdm"],
            check=True
        )
    except Exception as e:
        print("Error installing packages:", e)
    subprocess.run(["docker", "--version"], check=False)



## Downloading the OpenNeuro Dataset

> If you already have the dataset locally, skip this cell and set `BIDS_DIR` to your path.\n

In [ ]:
# ========== 2) Download ds002785 from OpenNeuro ==========
def download_dataset():
    # Requires datalad installed in system
    # subprocess.run(["pip", "install", "datalad"], check=True)
    subprocess.run(
        ["datalad", "install", "-s", "https://github.com/OpenNeuroDatasets/ds002785", "ds002785"],
        check=False
    )
    subprocess.run(["ls", "-la", "ds002785"], check=False)

## Running fMRIPrep for pre-processing in Docker

In [ ]:
# ========== 3) fMRIPrep command (Docker) ==========
def print_fmriprep_command(BIDS_DIR, OUT_DIR, WORK_DIR, FS_LICENSE, PARTICIPANTS):
    participant_flag = ''
    if len(PARTICIPANTS) > 0:
        participant_flag = '--participant-label ' + ' '.join(PARTICIPANTS)
    cmd = (
        f"docker run --rm -it "
        f"-v {BIDS_DIR}:/data:ro "
        f"-v {OUT_DIR}:/out "
        f"-v {WORK_DIR}:/work "
        f"-v {FS_LICENSE}:/opt/freesurfer/license.txt "
        f"nipreps/fmriprep:stable /data /out participant {participant_flag} "
        f"--fs-license-file /opt/freesurfer/license.txt "
        f"--output-spaces MNI152NLin2009cAsym:res-2 --use-aroma --skip-bids-validation"
    )

    print(cmd)

## Extracting features

In [ ]:



# ========== 4) Feature extraction ==========
def extract_features(BIDS_DIR, DERIV_DIR, SPACE):
    participants_tsv = os.path.join(BIDS_DIR, 'participants.tsv')
    if not os.path.exists(participants_tsv):
        raise FileNotFoundError(f'participants.tsv not found at {participants_tsv}')
    pheno = pd.read_csv(participants_tsv, sep='\t')

    label_col = None
    for c in ['diagnosis','label','group','social_anxiety','social_anxiety_diagnosis','clinical_diagnosis']:
        if c in pheno.columns:
            label_col = c
            break
    if label_col is None:
        print("Warning: could not auto-detect label column. Using dummy 'Control' labels.")
        pheno['Label'] = 'Control'
        label_col = 'Label'

    ho_cort = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
    ho_subc = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm')
    masker_cort = NiftiLabelsMasker(ho_cort.maps, standardize=True, detrend=True)
    masker_subc = NiftiLabelsMasker(ho_subc.maps, standardize=True, detrend=True)

    def find_preproc_bold(sub):
        pattern = os.path.join(DERIV_DIR, f'sub-{sub}', 'func', f'sub-{sub}_*{SPACE}*_desc-preproc_bold.nii.gz')
        return sorted(glob.glob(pattern))

    def labels_to_index_map(labels):
        return { (lbl.decode() if isinstance(lbl, bytes) else lbl): i for i,lbl in enumerate(labels) }

    cort_labels = labels_to_index_map(ho_cort.labels)
    subc_labels = labels_to_index_map(ho_subc.labels)

    def find_indices_by_names(mapping, names):
        idx = []
        for n in names:
            matches = [v for k,v in mapping.items() if n.lower() in k.lower()]
            idx.extend(matches)
        return sorted(set(idx))

    amyg_idx = find_indices_by_names(subc_labels, ['Amygdala'])
    pfc_idx  = find_indices_by_names(cort_labels, ['Frontal Pole','Frontal Medial','Superior Frontal','Middle Frontal','Orbital'])
    acc_idx  = find_indices_by_names(cort_labels, ['Anterior Cingulate'])
    pcc_idx  = find_indices_by_names(cort_labels, ['Posterior Cingulate'])
    mpfc_idx = find_indices_by_names(cort_labels, ['Frontal Medial','Frontal Pole','Superior Frontal Medial'])

    print('ROIs found (counts):', 'amyg', len(amyg_idx), 'pfc', len(pfc_idx), 'acc', len(acc_idx), 'pcc', len(pcc_idx), 'mpfc', len(mpfc_idx))

    subjects = [p.replace('sub-','') for p in pheno['participant_id'].astype(str)] if 'participant_id' in pheno.columns \
               else [r.split('_')[0].replace('sub-','') for r in os.listdir(DERIV_DIR) if r.startswith('sub-')]

    rows = []
    conn_estimator = ConnectivityMeasure(kind='correlation')

    for sub in subjects:
        bolds = find_preproc_bold(sub)
        if len(bolds) == 0:
            print(f'No preproc bold found for sub-{sub} - skipping')
            continue
        bold = bolds[0]
        conf_path = bold.replace('desc-preproc_bold.nii.gz','desc-confounds_regressors.tsv')
        confounds = None
        if os.path.exists(conf_path):
            try:
                confounds = pd.read_csv(conf_path, sep='\t')
            except Exception:
                confounds = None
        try:
            ts_cort = masker_cort.fit_transform(bold, confounds=confounds)
            ts_subc = masker_subc.fit_transform(bold, confounds=confounds)
        except Exception as e:
            print('Error extracting timeseries for', sub, e)
            continue

        def avg_signal(ts, idx_list):
            if len(idx_list) == 0:
                return None
            return np.nanmean(ts[:, idx_list], axis=1)

        amyg_signal = avg_signal(ts_subc, amyg_idx) if amyg_idx else None
        pfc_signal  = avg_signal(ts_cort, pfc_idx)  if pfc_idx else None
        acc_signal  = avg_signal(ts_cort, acc_idx)  if acc_idx else None
        pcc_signal  = avg_signal(ts_cort, pcc_idx)  if pcc_idx else None
        mpfc_signal = avg_signal(ts_cort, mpfc_idx) if mpfc_idx else None

        def safe_corr(a,b):
            if a is None or b is None: return np.nan
            if np.all(np.isfinite(a)) and np.all(np.isfinite(b)):
                if a.std()==0 or b.std()==0: return np.nan
                return float(np.corrcoef(a,b)[0,1])
            return np.nan

        amyg_pfc  = safe_corr(amyg_signal, pfc_signal)
        amyg_acc  = safe_corr(amyg_signal, acc_signal)
        pcc_mpfc  = safe_corr(pcc_signal, mpfc_signal)

        try:
            conn = conn_estimator.fit_transform([ts_cort])[0]
        except Exception as e:
            print('Error computing connectome for', sub, e)
            conn = None

        clustering_coeff = modularity = global_eff = np.nan
        if conn is not None:
            M = conn.copy()
            np.fill_diagonal(M,0)
            M_thr = np.where(M>0.2, M, 0)
            try:
                G = nx.from_numpy_array(M_thr)
                clustering_coeff = nx.average_clustering(G, weight='weight')
                comms = list(nx.algorithms.community.greedy_modularity_communities(G, weight='weight'))
                modularity = nx.algorithms.community.quality.modularity(G, comms, weight='weight')
                global_eff = nx.global_efficiency(G)
            except Exception as e:
                print('Graph metric error for', sub, e)

        pid = f'sub-{sub}'
        lab = pheno.loc[pheno['participant_id']==pid, label_col].values
        labv = lab[0] if len(lab)>0 else 'Unknown'

        rows.append({
            'Subject_ID': pid,
            'Amygdala_Prefrontal_corr': np.round(amyg_pfc, 3) if not np.isnan(amyg_pfc) else np.nan,
            'Amygdala_ACC_corr': np.round(amyg_acc, 3) if not np.isnan(amyg_acc) else np.nan,
            'PCC_mPFC_corr': np.round(pcc_mpfc, 3) if not np.isnan(pcc_mpfc) else np.nan,
            'Modularity': np.round(modularity, 3) if not np.isnan(modularity) else np.nan,
            'ClusteringCoeff': np.round(clustering_coeff, 3) if not np.isnan(clustering_coeff) else np.nan,
            'GlobalEfficiency': np.round(global_eff, 3) if not np.isnan(global_eff) else np.nan,
            'Label': labv
        })

    df_out = pd.DataFrame(rows)
    df_out.to_csv('secondary_dataset.csv', index=False, sep='\t')
    print(df_out.head())

## Main function to run the whole script

In [ ]:



# ==================== MAIN ====================
if __name__ == "__main__":
    # Step 1: install dependencies (optional if already installed)
    # install_requirements()

    # Step 2: download dataset (optional if already present)
    # download_dataset()

    # Step 3: print fMRIPrep Docker command
    print_fmriprep_command(
        BIDS_DIR='ds002785',
        OUT_DIR='derivatives/fmriprep',
        WORK_DIR='fmriprep_work',
        FS_LICENSE='~/.freesurfer/license.txt',
        PARTICIPANTS=[] # e.g., ['01','02']
    )

    # Step 4: feature extraction (requires fMRIPrep outputs)
    extract_features(
        BIDS_DIR='ds002785',
        DERIV_DIR='derivatives/fmriprep',
        SPACE='MNI152NLin2009cAsym'
    )



# **Deliverables 2** -


* Python Notebook for classification models.
* Documentation of methods and results.
* Automated update pipeline.






## Importing required packages and modules

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-import os

import json
import joblib
import glob
import warnings
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import (roc_auc_score, average_precision_score, precision_score, recall_score, f1_score,
                             accuracy_score, balanced_accuracy_score, confusion_matrix, roc_curve, precision_recall_curve)
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import hashlib
import time

warnings.filterwarnings('ignore')  # Suppressing warnings for cleaner output
RND = 42  # Random seed for reproducibility
OUT_DIR = Path('hda7_outputs')  # Output directory for results and models
OUT_DIR.mkdir(exist_ok=True)  # Create output directory if it doesn't exist

## Loading the .csv created by extracting features

In [2]:
def load_secondary(path='secondary_dataset.csv'):

    # Detects delimiter automatically (tab or comma).
    # Sets 'Subject_ID' column as index if present.
    df = pd.read_csv(path, sep='\t' if '\t' in open(path).read(1000) else ',')
    if 'Subject_ID' in df.columns:
        df = df.set_index('Subject_ID')
    return df

   ## Standardize and binarize labels into 'Case' (1) and 'Control' (0)
   ## Mapping done by searching keywords in label strings.
   ## Raises error if label column missing.




In [5]:
def sanitize_labels(df, label_col='Label'):
    if label_col not in df.columns:
      raise ValueError('Label column missing')
    df = df.copy()
    df[label_col] = df[label_col].astype(str)
    # Map labels containing 'social', 'case', or 'patient' to 'Case'
    mapping = {k: 'Case' for k in df[label_col].unique() if 'social' in k.lower() or 'case' in k.lower() or 'patient' in k.lower()}
    # Map labels containing 'control' or 'healthy' to 'Control'
    mapping.update({k: 'Control' for k in df[label_col].unique() if 'control' in k.lower() or 'healthy' in k.lower()})
    # Apply mapping; default to 'Control' if unmatched
    df['y'] = df[label_col].map(lambda x: mapping.get(x, 'Control'))
    df['y'] = (df['y'] == 'Case').astype(int)  # Convert to binary 0/1
    return df


 ## Extracting numeric features from Dataframe , dropping columns that are NaN or have null column names

In [6]:
def build_feature_table(df):
    feats = df.select_dtypes(include=[np.number]).copy()
    feats = feats.loc[:, feats.columns.notnull()]  # Drop columns with null names
    feats = feats.dropna(axis=1, how='all')  # Drop columns that are all NaN
    return feats

## Split features and labels into train/test and perfroming Nested Cross-validation

In [7]:
def train_test_split_subjectwise(X, y, test_size=0.2):

    # Uses fixed random seed.
    return train_test_split(X, y, test_size=test_size, stratify=y, random_state=RND)


def nested_cv_evaluate(X, y, estimator, param_grid, cv_outer=5, cv_inner=3):

    # Outer loop splits data for evaluation,
    # Inner loop performs hyperparameter tuning via GridSearchCV.

    # Returns list of dictionaries with performance metrics and best params per fold.

    outer = StratifiedKFold(n_splits=cv_outer, shuffle=True, random_state=RND)
    results = []
    for train_idx, test_idx in outer.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Pipeline: median imputation -> standard scaling -> classifier
        pipe = Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler()),
            ('clf', estimator)
        ])

        # Hyperparameter tuning with inner CV, scoring by ROC AUC
        gs = GridSearchCV(pipe, param_grid, cv=cv_inner, scoring='roc_auc', n_jobs=-1)
        gs.fit(X_train, y_train)

        best = gs.best_estimator_

        # Get predicted probabilities or decision function scores
        probas = best.predict_proba(X_test)[:,1] if hasattr(best, 'predict_proba') else best.decision_function(X_test)
        preds = best.predict(X_test)

        # Store metrics and predictions for this fold
        res = {
            'best_params': gs.best_params_,
            'roc_auc': roc_auc_score(y_test, probas),
            'pr_auc': average_precision_score(y_test, probas),
            'accuracy': accuracy_score(y_test, preds),
            'balanced_acc': balanced_accuracy_score(y_test, preds),
            'precision': precision_score(y_test, preds, zero_division=0),
            'recall': recall_score(y_test, preds, zero_division=0),
            'f1': f1_score(y_test, preds, zero_division=0),
            'y_test': y_test.values.tolist(),
            'y_score': probas.tolist(),
            'y_pred': preds.tolist()
        }
        results.append(res)
    return results

## Aggregate cross-validation results by computing mean and std for each metric across folds.

In [8]:
def aggregate_results(cv_results):

    metrics = ['roc_auc','pr_auc','accuracy','balanced_acc','precision','recall','f1']
    agg = {}
    for m in metrics:
        vals = [r[m] for r in cv_results]
        agg[m+'_mean'] = float(np.mean(vals))
        agg[m+'_std'] = float(np.std(vals))
    return agg

## Training the final model on the entire dataset using provided hyperparameters
## Compute permutation feature importance for trained model.

In [9]:

def train_final_model(X, y, estimator, params):


    # Applies median imputation and standard scaling in pipeline.

    pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
        ('clf', estimator)
    ])
    # Set classifier params (strip 'clf__' prefix if present)
    pipe.set_params(**{f'clf__{k}': v for k,v in params.items() if k.replace('clf__','')!=k})
    pipe.fit(X, y)
    return pipe


def permutation_imp(model, X, y, n_repeats=30):

   # Returns DataFrame sorted by importance.
    imp = permutation_importance(model, X, y, n_repeats=n_repeats, random_state=RND, n_jobs=-1)
    fi = pd.DataFrame({
        'feature': X.columns,
        'importance_mean': imp.importances_mean,
        'importance_std': imp.importances_std
    })
    fi = fi.sort_values('importance_mean', ascending=False)
    return fi

## Plotting various metrics and saving it in json files

In [10]:
def plot_roc_pr(y_test, y_score, out_prefix):

    # Plot and save ROC curve and Precision-Recall curve.

    fpr, tpr, _ = roc_curve(y_test, y_score)
    precision, recall, _ = precision_recall_curve(y_test, y_score)

    plt.figure()
    plt.plot(fpr, tpr)
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title('ROC')
    plt.savefig(f'{out_prefix}_roc.png')
    plt.close()

    plt.figure()
    plt.plot(recall, precision)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('PR')
    plt.savefig(f'{out_prefix}_pr.png')
    plt.close()


def plot_calibration(y_true, y_prob, out_path):

    # Plot calibration curve comparing predicted probabilities to observed frequencies.

    prob_true, prob_pred = calibration_curve(y_true, y_prob, n_bins=10)
    plt.figure()
    plt.plot(prob_pred, prob_true, marker='o')
    plt.plot([0,1],[0,1],'--')  # Reference line for perfect calibration
    plt.xlabel('Predicted probability')
    plt.ylabel('True probability')
    plt.title('Calibration')
    plt.savefig(out_path)
    plt.close()


def ablation_study(df, y, feature_groups):

    # Perform ablation study by training models on different feature subsets.
    # Uses RandomForestClassifier with nested CV.
    # Returns DataFrame with aggregated performance per feature group.

    ablation_results = []
    for name, cols in feature_groups.items():
        X_sub = df[cols].copy()
        X_sub = X_sub.select_dtypes(include=[np.number])
        X_sub = X_sub.fillna(X_sub.median())  # Fill missing values with median

        est = RandomForestClassifier(random_state=RND)
        params = {'clf__n_estimators':[100], 'clf__max_depth':[5,10]}
        res = nested_cv_evaluate(X_sub, y, est, params, cv_outer=5, cv_inner=3)
        agg = aggregate_results(res)
        agg['group'] = name
        ablation_results.append(agg)
    return pd.DataFrame(ablation_results)


def save_json(obj, path):

    # Save a Python object as a pretty-printed JSON file.

    with open(path,'w') as f:
        json.dump(obj, f, indent=2)

## Main Pipeline Function

In [11]:

def compute_file_hash(path):
    # Compute MD5 hash of a file for change detection.

    h = hashlib.md5()
    with open(path,'rb') as f:
        while chunk := f.read(8192):
            h.update(chunk)
    return h.hexdigest()


def run_pipeline(secondary_path='secondary_dataset.csv'):
    """
    Run the full pipeline end-to-end:
    - Load and sanitize data
    - Extract features and labels
    - Split data into train/test
    - Train and evaluate multiple classifiers with nested CV
    - Save CV results and aggregate metrics
    - Train final model with best parameters
    - Evaluate final model on test set and save metrics, plots
    - Compute feature importances and perform ablation study
    - Save comprehensive report with metadata and file hash
    """
    df = load_secondary(secondary_path)
    df = sanitize_labels(df, label_col='Label')
    feats = build_feature_table(df)
    y = df['y']
    feat_cols = feats.columns.tolist()

    # Split into train/test sets stratified by label
    X_train, X_test, y_train, y_test = train_test_split_subjectwise(feats, y, test_size=0.2)

    # Define classifiers to evaluate
    estimators = {
        'logreg': LogisticRegression(max_iter=1000, solver='liblinear'),
        'rf': RandomForestClassifier(random_state=RND),
        'svc': SVC(probability=True, random_state=RND)
    }

    # Hyperparameter grids for each classifier
    param_grids = {
        'logreg': {'clf__C':[0.01,0.1,1,10]},
        'rf': {'clf__n_estimators':[100,200], 'clf__max_depth':[5,10,None]},
        'svc': {'clf__C':[0.1,1,10], 'clf__kernel':['rbf','linear']}
    }

    # Nested CV for each estimator; save results and summary
    all_cv_results = {}
    for name, est in estimators.items():
        res = nested_cv_evaluate(feats, y, est, param_grids[name], cv_outer=5, cv_inner=3)
        agg = aggregate_results(res)
        all_cv_results[name] = agg
        save_json(res, OUT_DIR/f'cv_results_{name}.json')
    save_json(all_cv_results, OUT_DIR/'cv_summary.json')

    # Select best model based on mean ROC AUC
    best_model_name = max(all_cv_results.items(), key=lambda x: x[1]['roc_auc_mean'])[0]
    best_params = json.load(open(OUT_DIR/f'cv_results_{best_model_name}.json'))[0]['best_params']
    best_estimator = estimators[best_model_name]

    # Train final model on full dataset with best params
    fitted = train_final_model(feats, y, best_estimator, {k.replace('clf__',''):v for k,v in best_params.items()})
    joblib.dump(fitted, OUT_DIR/f'final_model_{best_model_name}.joblib')

    # Evaluate final model on holdout test set
    X_test_filled = X_test.fillna(X_test.median())  # Fill missing values in test set
    y_proba = fitted.predict_proba(X_test_filled)[:,1]
    y_pred = fitted.predict(X_test_filled)
    metrics = {
        'roc_auc': float(roc_auc_score(y_test, y_proba)),
        'pr_auc': float(average_precision_score(y_test, y_proba)),
        'accuracy': float(accuracy_score(y_test, y_pred)),
        'balanced_acc': float(balanced_accuracy_score(y_test, y_pred)),
        'precision': float(precision_score(y_test, y_pred, zero_division=0)),
        'recall': float(recall_score(y_test, y_pred, zero_division=0)),
        'f1': float(f1_score(y_test, y_pred, zero_division=0))
    }
    save_json(metrics, OUT_DIR/'final_test_metrics.json')

    # Plot ROC and PR curves
    plot_roc_pr(y_test, y_proba, OUT_DIR/f'final_{best_model_name}')
    # Plot calibration curve
    plot_calibration(y_test, y_proba, OUT_DIR/f'calibration_{best_model_name}.png')

    # Compute and save permutation feature importance
    fi = permutation_imp(fitted, feats, y)
    fi.to_csv(OUT_DIR/'permutation_importance.csv', index=False)

    # Define feature groups for ablation study
    feature_groups = {
        'amygdala_only':[c for c in feat_cols if 'Amygdala' in c or 'amyg' in c.lower()],
        'connectome_graph':['Modularity','ClusteringCoeff','GlobalEfficiency'],
        'all':feat_cols
    }
    ablation = ablation_study(feats, y, feature_groups)
    ablation.to_csv(OUT_DIR/'ablation_results.csv', index=False)

    # Save run summary report with metadata and file hash for reproducibility
    report = {
        'run': 'hda7 full pipeline',
        'random_seed': RND,
        'best_model': best_model_name,
        'metrics': metrics,
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
        'file_hash': compute_file_hash(secondary_path)
    }
    save_json(report, OUT_DIR/'report_summary.json')


def automated_update_pipeline(secondary_path='secondary_dataset.csv'):

    # Detect if the input dataset has changed by comparing file hashes.
    # If changed, run the full pipeline and update state file.
    # Otherwise, skip re-running the pipeline.

    state_file = OUT_DIR/'last_run.json'
    new_hash = compute_file_hash(secondary_path)
    old_hash = None
    if state_file.exists():
        old_state = json.load(open(state_file))
        old_hash = old_state.get('file_hash')
    if new_hash != old_hash:
        print("Detected change in dataset. Running full pipeline...")
        run_pipeline(secondary_path)
        save_json({'file_hash': new_hash, 'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')}, state_file)
    else:
        print("No changes in dataset. Skipping re-run.")

In [12]:
automated_update_pipeline('secondary_dataset.csv')


Detected change in dataset. Running full pipeline...
